In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [3]:
import requests
import json
import pandas as pd

excel_file = pd.read_excel('withNutrients.xlsx')
column_array = excel_file['Dietary intake'].values

carbs_col = []
calories_col = []
fiber_col = []
fat_col = []
protein_col = []


start_enrty = 950
end_enrty = 1000
# Define the range here
# first value is the starting index
# second value is the one after ending index(exclusive)
for i in range(start_enrty,end_enrty):

    query = column_array[i]
    
    url = 'https://trackapi.nutritionix.com/v2/natural/nutrients'
    headers = {
        #Add the API id and Key here
        'x-app-id': '2d640127',
        'x-app-key': '15e78cdde5473d4ccd1c1299749e0463',
        'x-remote-user-id': '0'
    }
    data = {'query': query}

    response = requests.post(url, headers=headers, data=data)
    data = response.json()

    with open('jsonFiles/entry'+str(i)+'.json', 'w') as f:
        json.dump(data, f)
    json_obj = json.loads(json.dumps(data))

    total_carbohydrates = 0
    total_calories = 0
    total_fiber = 0
    total_fat = 0
    total_protein = 0

    if "foods" in json_obj:
        for element in json_obj["foods"]:
            
            total_carbohydrates += (element['nf_total_carbohydrate'] or 0)
            total_calories += (element['nf_calories']or 0)
            total_fiber += (element['nf_dietary_fiber']or 0)
            total_fat += (element['nf_total_fat']or 0)
            total_protein += (element['nf_protein']or 0)


        carbs_col.append(total_carbohydrates)
        calories_col.append(total_calories)
        fiber_col.append(total_fiber)
        fat_col.append(total_fat)
        protein_col.append(total_protein)

        print("----------------Entry Details-  "+str(i)+"  --------------")
        print("")
        print("")
        print(column_array[i])
        print("")
        print("")
        print("carbs : "+str(total_carbohydrates))
        print("calories : "+str(total_calories))
        print("fiber : "+str(total_fiber))
        print("fat : "+str(total_fat))
        print("protein :"+str(total_protein))
        print("")
        print("")


----------------Entry Details-  950  --------------


Yogurt 100 g
Sweet potato 70 g


carbs : 21.54
calories : 126
fiber : 2.31
fat : 1.6600000000000001
protein :6.66


----------------Entry Details-  951  --------------


Vegetable 100 g
Shredded radish 50 g
Broccoli, dried bean curd and shredded carrot 100 g
Coarse grain 70 g


carbs : 21.259999999999998
calories : 111.81
fiber : 7.07
fat : 1.25
protein :4.93


----------------Entry Details-  952  --------------


Black rice cake 60 g
Egg 60 g
Peanuts with vinegar 40 g
Soybean milk 250 ml


carbs : 95.19
calories : 690.6299999999999
fiber : 0.91
fat : 23.75
protein :22.54


----------------Entry Details-  953  --------------


Stir fried vegetable 130 g
Broccoli, dried bean, carrot and shredded pork 100 g
Vegetable ball 150 g


carbs : 76.41
calories : 649.24
fiber : 23.64
fat : 19.56
protein :42.62


----------------Entry Details-  954  --------------


Yogurt 100 g
Sweet potato 100 g


carbs : 27.75
calories : 153
fiber : 3.3
fat 

In [4]:
import openpyxl

# Load the existing workbook
workbook = openpyxl.load_workbook("withNutrients.xlsx")

# Get the active worksheet
worksheet = workbook.active

starting_row = start_enrty

for index, value in enumerate(carbs_col, start=starting_row+2):
    worksheet.cell(row=index, column=10, value=value)

for index, value in enumerate(calories_col, start=starting_row+2):
    worksheet.cell(row=index, column=11, value=value)

for index, value in enumerate(fiber_col, start=starting_row+2):
    worksheet.cell(row=index, column=12, value=value)

for index, value in enumerate(fat_col, start=starting_row+2):
    worksheet.cell(row=index, column=13, value=value)

for index, value in enumerate(protein_col, start=starting_row+2):
    worksheet.cell(row=index, column=14, value=value)

workbook.save("withNutrients.xlsx")
